In [ ]:
import requests
from xml.etree import ElementTree as ET
import pandas as pd

In [ ]:
madd_request_xml = '''<?xml version="1.0" encoding="UTF-8"?>
<eCH-0206:maddRequest xmlns:eCH-0058="http://www.ech.ch/xmlns/eCH-0058/5" xmlns:eCH-0206="http://www.ech.ch/xmlns/eCH-0206/2" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.ech.ch/xmlns/eCH-0206/2 eCH-0206-2-0.xsd">
     <eCH-0206:requestHeader>
   	   	<eCH-0206:messageId>1714222364875</eCH-0206:messageId>
   	   	<eCH-0206:businessReferenceId>1714222364875</eCH-0206:businessReferenceId>
   	   	<eCH-0206:requestingApplication>
   	   	   	<eCH-0058:manufacturer>FSO</eCH-0058:manufacturer>
   	   	   	<eCH-0058:product>MADDAssist</eCH-0058:product>
   	   	   	<eCH-0058:productVersion>1.0.0</eCH-0058:productVersion>
   	   	</eCH-0206:requestingApplication>
   	   	<eCH-0206:requestDate>2024-04-27T14:52:44</eCH-0206:requestDate>
   	</eCH-0206:requestHeader>
   	<eCH-0206:requestContext>building</eCH-0206:requestContext>
   	<eCH-0206:requestQuery>
   	   	<eCH-0206:condition>
            	<eCH-0206:attributePath>/eCH-0206:maddResponse/eCH-0206:buildingList/eCH-0206:buildingItem/eCH-0206:building/eCH-0206:buildingCategory</eCH-0206:attributePath>
            	<eCH-0206:operator>equalTo</eCH-0206:operator>
            	<eCH-0206:attributeValue>1020</eCH-0206:attributeValue>
        	</eCH-0206:condition>
   	   	<eCH-0206:condition>
            	<eCH-0206:attributePath>/eCH-0206:maddResponse/eCH-0206:buildingList/eCH-0206:buildingItem/eCH-0206:building/eCH-0206:buildingStatus</eCH-0206:attributePath>
            	<eCH-0206:operator>equalTo</eCH-0206:operator>
            	<eCH-0206:attributeValue>1007</eCH-0206:attributeValue>
        	</eCH-0206:condition>
   	   	<eCH-0206:condition>
            	<eCH-0206:attributePath>/eCH-0206:maddResponse/eCH-0206:buildingList/eCH-0206:buildingItem/eCH-0206:municipality/eCH-0206:municipalityId</eCH-0206:attributePath>
            	<eCH-0206:operator>equalTo</eCH-0206:operator>
            	<eCH-0206:attributeValue>261</eCH-0206:attributeValue>
        	</eCH-0206:condition>
   	</eCH-0206:requestQuery>
</eCH-0206:maddRequest>
'''


In [ ]:
# URL der MADD-API
url = 'https://madd.bfs.admin.ch/eCH-0206'

# Setze die benötigten Header, oft ist Content-Type wichtig
headers = {
    'Content-Type': 'text/xml'  # oder 'text/xml' je nach API
}


In [ ]:
# Senden der POST-Anfrage mit dem XML-Inhalt
response = requests.post(url, data=madd_request_xml, headers=headers)

# Überprüfen, ob die Anfrage erfolgreich war
if response.status_code == 200:
    print("Anfrage erfolgreich!")
    # Hier kannst du die Antwortdaten ausgeben oder weiterverarbeiten
    print(response.text)
else:
    print("Anfrage fehlgeschlagen:", response.status_code)
    print(response.text)

In [ ]:
# Definiere den Pfad und den Namen der Datei, in der die XML-Daten gespeichert werden sollen
file_path = 'abgebrochene_wohnungen.xml'

# Öffne die Datei im Schreibmodus und speichere die XML-Daten
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(response.text)  # Ersetze 'response.text' mit der Variablen, die deine XML-Daten enthält, wenn sie anders benannt ist

print(f'Daten erfolgreich in {file_path} gespeichert!')


In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd

def xml_to_csv(xml_filename, csv_filename):
    ns = {'ns': 'http://www.ech.ch/xmlns/eCH-0206/2'}  # Namespace declaration

    tree = ET.parse(xml_filename)
    root = tree.getroot()

    data = []
    for building in root.findall('.//ns:buildingItem', ns):
        building_data = building.find('.//ns:building', ns)

        # Extract basic data points directly within <building>
        egid = building.find('.//ns:EGID', ns).text if building.find('.//ns:EGID', ns) is not None else 'N/A'
        officialBuildingNo = building_data.find('ns:officialBuildingNo', ns).text if building_data.find('ns:officialBuildingNo', ns) is not None else 'N/A'
        buildingStatus = building_data.find('ns:buildingStatus', ns).text if building_data.find('ns:buildingStatus', ns) is not None else 'N/A'
        buildingCategory = building_data.find('ns:buildingCategory', ns).text if building_data.find('ns:buildingCategory', ns) is not None else 'N/A'
        buildingClass = building_data.find('ns:buildingClass', ns).text if building_data.find('ns:buildingClass', ns) is not None else 'N/A'
        
        # Extracting data for 'dateOfConstruction' and handle nested details
        date_construction = building_data.find('.//ns:dateOfConstruction', ns)
        if date_construction is not None:
            date_of_construction = date_construction.find('.//ns:dateOfConstruction', ns).text if date_construction.find('.//ns:dateOfConstruction', ns) is not None else 'N/A'
        else:
            date_of_construction = 'N/A'

        yearOfDemolition = building_data.find('ns:yearOfDemolition', ns).text if building_data.find('ns:yearOfDemolition', ns) is not None else 'N/A'
        surfaceAreaOfBuilding = building_data.find('ns:surfaceAreaOfBuilding', ns).text if building_data.find('ns:surfaceAreaOfBuilding', ns) is not None else 'N/A'
        numberOfFloors = building_data.find('ns:numberOfFloors', ns).text if building_data.find('ns:numberOfFloors', ns) is not None else 'N/A'

        # Collect all values in 'thermotechnicalDeviceForHeating1'
        heating_elements = building_data.findall('.//ns:thermotechnicalDeviceForHeating1/*', ns)
        heating_info = ', '.join([f"{elem.tag.split('}')[-1]}: {elem.text}" for elem in heating_elements if elem.text]) if heating_elements else 'N/A'

        # Collect all values in 'recordModification'
        record_mod_elements = building_data.findall('.//ns:recordModification/*', ns)
        record_mod_info = ', '.join([f"{elem.tag.split('}')[-1]}: {elem.text}" for elem in record_mod_elements if elem.text]) if record_mod_elements else 'N/A'

        data.append({
            'EGID': egid,
            'Official Building No': officialBuildingNo,
            'Building Status': buildingStatus,
            'Building Category': buildingCategory,
            'Building Class': buildingClass,
            'Date of Construction': date_of_construction,
            'Year of Demolition': yearOfDemolition,
            'Surface Area of Building': surfaceAreaOfBuilding,
            'Number of Floors': numberOfFloors,
            'Heating Device Info': heating_info,
            'Record Modification Info': record_mod_info
        })

    df = pd.DataFrame(data)
    df.to_csv(csv_filename, index=False)
    return f'Data successfully saved in {csv_filename}'

# File paths
xml_filename = '/Users/mariusaffolter/Documents/200_Studium/220_Semester/24FS/Einsatz Geodaten Marketing/Project/apartment_analysis_egm/abgebrochene_wohnungen.xml'
csv_filename = '/Users/mariusaffolter/Documents/200_Studium/220_Semester/24FS/Einsatz Geodaten Marketing/Project/apartment_analysis_egm/abgebrochene_wohnungen.csv'
result = xml_to_csv(xml_filename, csv_filename)
print(result)
